# Data Analysis - Cleaning Data
### Loans Originated in 2018
### Predict/Classifying what types of loans/borrowers will be 

Create Path to Import *.py files (not needed in "index.ipynb")

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Combined all below cells for data and save to CSV in a "preprocess" function. <br>
Uncomment and run only if needed, as the edited and final of the codes from this notebook to create a cleaned dataset is in the "preprocess.py"

In [2]:
# from codes.preprocess import preprocess
# preprocess()

#### Import Libraries

In [31]:
import time
from codes.preprocess import read_csv
import pandas as pd

TypeError: 'DictWrapper' object is not callable

Time to load "Acquisition":  4 to 5 seconds
<br>Time to load "Performance":  40 to 50 seconds

In [5]:
start_time = time.time()
acquisition_df = read_csv('Acquisition')
print(f'Time to load "Acquisition" {round(time.time()-start_time,4)} seconds')

Time to load "Acquisition" 4.9923 seconds


In [6]:
start_time = time.time()
performance_df = read_csv('Performance')
print(f'Time to load "Performance" {round(time.time()-start_time,4)} seconds')

Time to load "Performance" 51.5609 seconds


In [7]:
display(acquisition_df.head(5))
display(performance_df.head(5))

,id,channel,seller,interest_rate,org_balance,loan_term,origination_date,ltv,cltv,borrower_count,dti,borrower_score,first_time_homebuyer,loan_purpose,property_type,unit_count,occupancy_type,property_state,insurance_percentage,product_type,coborrower_score,insurance_type,relocation_flag
0,100001040173,R,QUICKEN LOANS INC.,4.25,453000,360,01/2018,65,65.0,1,28.0,791.0,N,C,PU,1,P,OH,NaN,FRM,NaN,NaN,N
1,100002370993,C,"WELLS FARGO BANK, N.A.",4.25,266000,360,01/2018,80,80.0,2,41.0,736.0,N,R,PU,1,P,IN,NaN,FRM,793.0,NaN,N
2,100005405807,R,PMTT4,3.99,233000,360,12/2017,79,79.0,2,48.0,696.0,N,R,SF,1,P,CA,NaN,FRM,665.0,NaN,N
3,100008071646,R,OTHER,4.25,184000,360,01/2018,80,80.0,1,48.0,767.0,Y,P,PU,1,P,FL,NaN,FRM,NaN,NaN,N
4,100010739040,R,OTHER,4.25,242000,360,02/2018,49,49.0,1,22.0,727.0,N,R,SF,1,P,CA,NaN,FRM,NaN,NaN,N


,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag
0,100001040173,02/01/2018,4.25,NaN,0,360,360.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
1,100001040173,03/01/2018,4.25,NaN,1,359,359.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
2,100001040173,04/01/2018,4.25,NaN,2,358,358.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
3,100001040173,05/01/2018,4.25,NaN,3,357,357.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
4,100001040173,06/01/2018,4.25,NaN,4,356,356.0,18140,0,N,NaN,NaN,NaN,NaN,NaN


## SQLite DataBase
NOTE: Uncomment to creat a SQLite Database on computer

In [8]:
# import sqlite3
# conn = sqlite3.connect('Secondary_Mortgage_Loans.db')
# acquisition_df

In [9]:
# acquisition_df.to_sql('Acquisition', con=conn)
# performance_df.to_sql('Performance', con=conn)

## EDA

In [10]:
acquisition_df.info()
performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625195 entries, 0 to 1625194
Data columns (total 23 columns):
id                      1625195 non-null int64
channel                 1625195 non-null object
seller                  1625195 non-null object
interest_rate           1625195 non-null float64
org_balance             1625195 non-null int64
loan_term               1625195 non-null int64
origination_date        1625195 non-null object
ltv                     1625195 non-null int64
cltv                    1625194 non-null float64
borrower_count          1625195 non-null int64
dti                     1624702 non-null float64
borrower_score          1623501 non-null float64
first_time_homebuyer    1625195 non-null object
loan_purpose            1625195 non-null object
property_type           1625195 non-null object
unit_count              1625195 non-null int64
occupancy_type          1625195 non-null object
property_state          1625195 non-null object
insurance_percentage    5

#### Filter rows outside of 50 states + DC. (2298 rows)
In both Acquisition and Performance.
- Use Acquisition's Loan ID to filter out rows in Performance.

Drop State Codes with 'PR','GU', and 'VI' from both Acquisition and Performance.

In [11]:
acquisition_df.property_state = acquisition_df.property_state.where(~acquisition_df.property_state.isin(['PR','GU','VI']))
acquisition_df = acquisition_df.dropna(subset=['property_state'])
acquisition_df.shape

(1622897, 23)

In [12]:
print('Before removing: ', performance_df.shape[0])
performance_df.id = performance_df.id.where(performance_df.id.isin(acquisition_df.id))
performance_df = performance_df.dropna(subset=['id'])
print('After removing: ', performance_df.shape[0])

Before removing:  24459263
After removing:  24423998


Change "Delinquency Status" type (string to integers) and change "Reporting Period" as DateTime

In [13]:
performance_df['reporting_period']= pd.to_datetime(performance_df['reporting_period'])

In [14]:
performance_df.delinquency_status = performance_df.delinquency_status.fillna(-99).mask(performance_df.delinquency_status == 'X',-99).astype('int8')

In [15]:
# print(performance_df.delinquency_status.isna().sum(), ' rows of NaN in "Delinquency Status"')
performance_df['delinquency_status'].value_counts()

 0     24022577
-99      222653
 1       131824
 2        21958
 3         8630
 4         5127
 5         3621
 6         2367
 7         1620
 8         1173
 9          797
 10         593
 11         405
 12         270
 13         164
 14          96
 15          60
 16          35
 17          17
 18           7
 19           4
Name: delinquency_status, dtype: int64

In [16]:
performance_df['delinquency_bool'] = performance_df.delinquency_status.map(lambda x: 1 if x > 0 else 0)

In [18]:
performance_df['payment_amounts'] = -performance_df.groupby('id').upc_balance.diff()

Backfill NaN for Balance for the first 5-6 months

In [19]:
performance_df.upc_balance = performance_df.upc_balance.fillna(method='bfill')

In [20]:
performance_df = performance_df[performance_df.loan_age > 0]

In [21]:
performance_df.head(5)

,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
1,1.000010e+11,2018-03-01,4.25,449321.18,1,359,359.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
2,1.000010e+11,2018-04-01,4.25,449321.18,2,358,358.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
3,1.000010e+11,2018-05-01,4.25,449321.18,3,357,357.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
4,1.000010e+11,2018-06-01,4.25,449321.18,4,356,356.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
5,1.000010e+11,2018-07-01,4.25,449321.18,5,355,355.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN


In [ ]:
# zero_balanced = performance_df[performance_df.zero_balance_code==1]

In [22]:
performance_df[performance_df.upc_balance == 0]

,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
6928525,8.574486e+11,2019-05-01,4.375,0.0,13,347,0.0,44100,-99,N,1.0,05/2019,NaN,NaN,N,0,259645.96
11274145,5.429086e+11,2019-04-01,4.875,0.0,10,350,0.0,0,-99,N,1.0,04/2019,NaN,NaN,N,0,181445.91
15191139,2.105369e+11,2019-08-01,3.925,0.0,11,169,0.0,17460,-99,N,6.0,08/2019,NaN,NaN,Y,0,111570.29


In [23]:
performance_df.zero_balance_code.value_counts()

1.0     220908
6.0       1116
9.0        143
2.0         43
3.0         19
15.0         8
16.0         1
Name: zero_balance_code, dtype: int64

Looking at some delinquency_status data.

Example of 19 months (max number) of Deliquent Loan

In [24]:
performance_df[performance_df.id == 598653679058].tail(5)

,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
4563853,5.986537e+11,2019-05-01,5.5,142344.02,16,344,359.0,26420,15,N,NaN,NaN,NaN,NaN,NaN,1,-0.0
4563854,5.986537e+11,2019-06-01,5.5,142344.02,17,343,359.0,26420,16,N,NaN,NaN,NaN,NaN,NaN,1,-0.0
4563855,5.986537e+11,2019-07-01,5.5,142344.02,18,342,359.0,26420,17,N,NaN,NaN,NaN,NaN,NaN,1,-0.0
4563856,5.986537e+11,2019-08-01,5.5,142344.02,19,341,359.0,26420,18,N,NaN,NaN,NaN,NaN,NaN,1,-0.0
4563857,5.986537e+11,2019-09-01,5.5,142344.02,20,340,359.0,26420,19,N,NaN,NaN,NaN,NaN,NaN,1,-0.0


### Fully-paid \[zero_balance_code = 1; make_whole_flag (Repurchase Make Whole Proceeds Flag) = 'N'\]
### RealEstateOwned \[zero_balance_code = 9 \]

In [25]:
fully_paid = performance_df[(performance_df.zero_balance_code.isin([1, 9])) & (performance_df.make_whole_flag=='N')]

### Foreclosure \[zero_balance_code = 3,6,15]

In [26]:
defaulted = performance_df[performance_df.zero_balance_code.isin([3,6,15])]

### DELINQUENT

In [27]:
delinq_ids = performance_df[performance_df['delinquency_bool']==1].groupby('id').nth(0).reset_index().id

In [28]:
delinq_loans = performance_df[performance_df.id.isin(delinq_ids)]

In [29]:
display(delinq_loans.head(5))
display(delinq_loans.tail(5))

,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
319,1.000316e+11,2018-05-01,4.375,365255.06,1,359,359.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
320,1.000316e+11,2018-06-01,4.375,365255.06,2,358,357.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
321,1.000316e+11,2018-07-01,4.375,365255.06,3,357,357.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
322,1.000316e+11,2018-08-01,4.375,365255.06,4,356,356.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
323,1.000316e+11,2018-09-01,4.375,365255.06,5,355,355.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN


,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
24459104,9.999634e+11,2019-05-01,5.25,276631.01,6,354,353.0,31460,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
24459105,9.999634e+11,2019-06-01,5.25,276631.01,7,353,353.0,31460,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
24459106,9.999634e+11,2019-07-01,5.25,276301.17,8,352,352.0,31460,0,N,NaN,NaN,NaN,NaN,NaN,0,329.84
24459107,9.999634e+11,2019-08-01,5.25,276301.17,9,351,352.0,31460,1,N,NaN,NaN,NaN,NaN,NaN,1,-0.00
24459108,9.999634e+11,2019-09-01,5.25,275302.97,10,350,349.0,31460,0,N,NaN,NaN,NaN,NaN,NaN,0,998.20


In [30]:
before_deliq_rows = delinq_loans[delinq_loans.groupby('id').delinquency_status.diff(-1) == -1].groupby('id').nth(0)
before_deliq_rows.head(5)

,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
id,,,,,,,,,,,,,,,,
1.000028e+11,2018-06-01,4.750,331674.96,2,358,358.0,19100,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
1.000316e+11,2018-11-01,4.375,364094.15,7,353,353.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,581.51
1.000533e+11,2018-12-01,4.625,133990.26,5,355,355.0,0,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
1.000666e+11,2019-02-01,5.375,346032.71,3,357,357.0,29820,0,N,NaN,NaN,NaN,NaN,NaN,0,NaN
1.000754e+11,2019-05-01,4.500,57133.36,12,168,168.0,19740,0,N,NaN,NaN,NaN,NaN,NaN,0,243.84


<p>behavior of payers up to point of deliquency
<br>(using balance)
</p>

#### Delinq_status = 1, but use the delinq_status = 0 row

In [37]:
before_deliq_rows = performance_df[(performance_df['delinquency_status']==0) & 
                                   (performance_df.id.isin(delinq_ids))].groupby('id').nth(-1).reset_index()
before_deliq_rows.head()

,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
0,1.000028e+11,2019-09-01,4.750,326555.37,17,343,341.0,19100,0,N,NaN,NaN,NaN,NaN,NaN,0,453.11
1,1.000316e+11,2019-09-01,4.375,345644.81,17,343,319.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0,1042.09
2,1.000533e+11,2019-09-01,4.625,132187.71,14,346,345.0,0,0,N,NaN,NaN,NaN,NaN,NaN,0,366.08
3,1.000666e+11,2019-09-01,5.375,344020.97,10,350,350.0,29820,0,N,NaN,NaN,NaN,NaN,NaN,0,810.09
4,1.000754e+11,2019-09-01,4.500,56148.84,16,164,164.0,19740,0,N,NaN,NaN,NaN,NaN,NaN,0,247.51


In [38]:
not_delinq = performance_df[~performance_df.id.isin(before_deliq_rows.id)]

In [39]:
defaulted = performance_df[~performance_df.foreclosure_date.isna()]

### Current \[delinquency_status = 0]

In [40]:
current_loans = performance_df[performance_df.delinquency_status==0].groupby('id').nth(-1).reset_index()

In [41]:
current_loans[~current_loans.id.isin(delinq_ids) | ~current_loans.id.isin(defaulted) | ~current_loans.id.isin(
        fully_paid)]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
0,1.000003e+11,2019-09-01,4.250,262004.04,15,345,336.0,19780,0,N,NaN,NaN,NaN,NaN,NaN,0,639.76
1,1.000010e+11,2019-09-01,4.250,440853.48,19,341,341.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,665.27
2,1.000016e+11,2019-03-01,4.625,337063.04,11,349,349.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0,457.50
3,1.000016e+11,2019-08-01,4.625,446559.21,11,349,349.0,16980,0,N,NaN,NaN,NaN,NaN,NaN,0,-0.00
4,1.000022e+11,2019-09-01,5.000,84065.32,8,352,351.0,26420,0,N,NaN,NaN,NaN,NaN,NaN,0,105.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619761,9.999979e+11,2019-09-01,4.375,279782.33,12,348,348.0,22220,0,N,NaN,NaN,NaN,NaN,NaN,0,398.85
1619762,9.999985e+11,2019-09-01,5.500,392477.89,16,344,343.0,41740,0,N,NaN,NaN,NaN,NaN,NaN,0,471.29
1619763,9.999985e+11,2019-09-01,4.875,67477.76,17,283,282.0,37460,0,N,NaN,NaN,NaN,NaN,NaN,0,127.75
1619764,9.999988e+11,2019-09-01,4.250,348713.11,19,341,341.0,39900,0,N,NaN,NaN,NaN,NaN,NaN,0,526.22


In [44]:
current_loans = current_loans[~current_loans.id.isin(before_deliq_rows.id)]
current_loans = current_loans[~current_loans.id.isin(defaulted.id)]
current_loans.groupby('id').nth(-1)

,reporting_period,interest_rate,upc_balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool,payment_amounts
id,,,,,,,,,,,,,,,,
1.000003e+11,2019-09-01,4.250,262004.04,15,345,336.0,19780,0,N,NaN,NaN,NaN,NaN,NaN,0,639.76
1.000010e+11,2019-09-01,4.250,440853.48,19,341,341.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0,665.27
1.000016e+11,2019-03-01,4.625,337063.04,11,349,349.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0,457.50
1.000016e+11,2019-08-01,4.625,446559.21,11,349,349.0,16980,0,N,NaN,NaN,NaN,NaN,NaN,0,-0.00
1.000022e+11,2019-09-01,5.000,84065.32,8,352,351.0,26420,0,N,NaN,NaN,NaN,NaN,NaN,0,105.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.999979e+11,2019-09-01,4.375,279782.33,12,348,348.0,22220,0,N,NaN,NaN,NaN,NaN,NaN,0,398.85
9.999985e+11,2019-09-01,5.500,392477.89,16,344,343.0,41740,0,N,NaN,NaN,NaN,NaN,NaN,0,471.29
9.999985e+11,2019-09-01,4.875,67477.76,17,283,282.0,37460,0,N,NaN,NaN,NaN,NaN,NaN,0,127.75
